In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../Le_Son_LV2_Project_01')
from confluent_kafka import Producer
from config import load_config
from random import choice

In [2]:
kafka_config = load_config(filename="database.ini", section="local_kafka")
kafka_config

{'servers': 'localhost:9094,localhost:9194,localhost:9294',
 'username': 'kafka',
 'password': 'UnigapKafka@2024',
 'default_consumer_group_id': 'sonhaile-connection',
 'default_consumer_topic': 'product_view',
 'default_producer_topic': 'product_view',
 'default_producer_acks': 'all',
 'default_producer_min_isr': '2'}

In [ ]:
if __name__ == '__main__':

    config = {
            'bootstrap.servers': kafka_config['servers'],
            'security.protocol': 'SASL_PLAINTEXT',       # or 'SASL_SSL' if using SSL
            'sasl.mechanism': 'PLAIN',
            'sasl.username': kafka_config['username'],
            'sasl.password': kafka_config['password'],
            'acks': 'all'
            }

    # Create Producer instance
    producer = Producer(config)

    # Optional per-message delivery callback (triggered by poll() or flush())
    # when a message has been successfully delivered or permanently
    # failed delivery (after retries).
    def delivery_callback(err, msg):
        if err:
            print('ERROR: Message failed delivery: {}'.format(err))
        else:
            print("Produced event to topic {topic}: key = {key:12} value = {value:12}".format(
                topic=msg.topic(), key=msg.key().decode('utf-8'), value=msg.value().decode('utf-8')))

    # Produce data by selecting random values from these lists.
    topic = "test_topic_sonhaile"
    user_ids = ['eabara', 'jsmith', 'sgarcia', 'jbernard', 'htanaka', 'awalther']
    products = ['book', 'alarm clock', 't-shirts', 'gift card', 'batteries']

    count = 0
    for _ in range(10):
        user_id = choice(user_ids)
        product = choice(products)
        producer.produce(topic = topic, value = product, key = user_id, callback=delivery_callback)
        count += 1

    # Block until the messages are sent.
    producer.poll(10000)
    producer.flush()

Produced event to topic test_topic_sonhaile: key = jsmith       value = batteries   
Produced event to topic test_topic_sonhaile: key = jsmith       value = gift card   
Produced event to topic test_topic_sonhaile: key = jsmith       value = gift card   
Produced event to topic test_topic_sonhaile: key = sgarcia      value = batteries   
Produced event to topic test_topic_sonhaile: key = awalther     value = batteries   
Produced event to topic test_topic_sonhaile: key = eabara       value = t-shirts    
Produced event to topic test_topic_sonhaile: key = eabara       value = gift card   
Produced event to topic test_topic_sonhaile: key = jbernard     value = book        
Produced event to topic test_topic_sonhaile: key = eabara       value = book        
Produced event to topic test_topic_sonhaile: key = jbernard     value = batteries   
